In [1]:
import numpy as np
import pandas as pd

In [2]:
# TRSC = "流水数据/table_loss_transaction_trainingdata.txt"
TRSC = "流水数据/table_loss_transaction_testdata.txt"
# TP = "抛出点数据/table_throwpoint_training.txt"
TP = "抛出点数据/table_throwpoint_test.txt" 

M = "商户数据/table_datamining_merchant.txt"
# LBL = "特征值/lable.csv"
# TRSC = "特征值/trsc_cut.csv"
# EIGEN = "特征值/eigen_train.csv"
EIGEN = "特征值/eigen_test.csv"

In [3]:
# try:
#     trsc
# except NameError:
#     trsc_exists = False
#     trsc=pd.read_csv(TRSC,engine='python')
# else:
#     trsc_exists = True
# trsc
trsc=pd.read_csv(TRSC,engine='python')

In [4]:
trsc.drop(columns=['id','terminal_id','payer_uid','received_amount','paid_amount','bankcard_credit','wallet_alipay_finance',
                   'alipay_point','hongbao_wosai','hongbao_wosai_mch','discount_wosai','discount_wosai_mch','hongbao_channel_mch',
                   'hongbao_channel_mch_top_up','card_pre','card_balance','merchant_sn','is_liquidation_next_day','type',
                   'status','pay_way','sub_pay_way','ctime'],inplace=True)

## 特征1：#original_amount

## 特征2：（支付通道中）是否优惠

price_reduced = bool(#original_amount - #effective_amount)

price_reduced = 1 : 有优惠

price_reduced = 0 : 无优惠

In [5]:
trsc['price_reduced']= trsc['original_amount']-trsc['effective_amount']

In [6]:
#确认没有负值
for i in trsc.price_reduced:
    if i<0:
        print("@")

In [7]:
trsc['if_price_reduced'] = trsc['price_reduced'].astype(bool).astype(int)

In [8]:
 sum(trsc.if_price_reduced)

865

## 特征3： 提供的支付方式个数

（4*1vector) 

#payw_offer = sum[bool(bankcard_debit), bool(wallet_weixin), bool(wallet_alipay), bool(alipay_huabei)]


In [9]:
payw_sum = trsc.groupby(['store_id']).mean().reset_index()
payw_sum['bankcard_debit'] = payw_sum['bankcard_debit'].astype(bool).astype(int)
payw_sum['wallet_alipay'] = payw_sum['wallet_alipay'].astype(bool).astype(int)
payw_sum['wallet_weixin'] = payw_sum['wallet_weixin'].astype(bool).astype(int)
payw_sum['alipay_huabei'] = payw_sum['alipay_huabei'].astype(bool).astype(int)
payw_sum['payw_offer']=payw_sum['bankcard_debit']+payw_sum['wallet_alipay']+payw_sum['wallet_weixin']+payw_sum['alipay_huabei']
payw_sum
trsc = trsc.merge(payw_sum[['store_id','payw_offer']],on='store_id')

In [10]:
trsc.drop(columns=['effective_amount','bankcard_debit','wallet_weixin','wallet_alipay','alipay_huabei'],inplace=True)

## 特征4：（商户提供的）活动

bool(discount_channel & discount_channel &       )

In [11]:
payw_sum['discount']=payw_sum['discount_channel'].astype(bool).astype(int)+payw_sum['discount_channel_mch'].astype(bool).astype(int)+payw_sum['hongbao_channel'].astype(bool).astype(int)+payw_sum['discount_channel_mch_top_up'].astype(bool).astype(int)
trsc = trsc.merge(payw_sum[['store_id','discount']],on='store_id')

In [12]:
payw_sum['if_discount']=payw_sum['discount_channel'].astype(bool)+payw_sum['discount_channel_mch'].astype(bool)+payw_sum['hongbao_channel'].astype(bool)+payw_sum['discount_channel_mch_top_up'].astype(bool)
sum(payw_sum.if_discount)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\computation\expressions.py:183: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  .format(op=op_str, alt_op=unsupported[op_str]))


2270

In [13]:
trsc.drop(columns=['discount_channel','discount_channel_mch','discount_channel_mch_top_up','hongbao_channel'],inplace=True)

In [14]:
trsc

,store_id,merchant_id,original_amount,pt,price_reduced,if_price_reduced,payw_offer,discount
0,008b842b-34d7-4810-adf3-82e7fc75d355,cee01785-713d-4c89-9912-ec83fa749c43,800,20180512,0,0,4,2
1,008b842b-34d7-4810-adf3-82e7fc75d355,cee01785-713d-4c89-9912-ec83fa749c43,800,20180512,0,0,4,2
2,008b842b-34d7-4810-adf3-82e7fc75d355,cee01785-713d-4c89-9912-ec83fa749c43,2200,20180512,0,0,4,2
3,008b842b-34d7-4810-adf3-82e7fc75d355,cee01785-713d-4c89-9912-ec83fa749c43,800,20180512,0,0,4,2
4,008b842b-34d7-4810-adf3-82e7fc75d355,cee01785-713d-4c89-9912-ec83fa749c43,800,20180408,0,0,4,2
5,008b842b-34d7-4810-adf3-82e7fc75d355,cee01785-713d-4c89-9912-ec83fa749c43,800,20180512,0,0,4,2
6,008b842b-34d7-4810-adf3-82e7fc75d355,cee01785-713d-4c89-9912-ec83fa749c43,1600,20180519,0,0,4,2
7,008b842b-34d7-4810-adf3-82e7fc75d355,cee01785-713d-4c89-9912-ec83fa749c43,800,20180421,0,0,4,2
8,008b842b-34d7-4810-adf3-82e7fc75d355,cee01785-713d-4c89-9912-ec83fa749c43,500,20180517,0,0,4,2
9,008b842b-34d7-4810-adf3-82e7fc75d355,cee01785-713d-4c89-9912-ec83fa749c43,1400,20180407,0,0,4,2


## 不看throwpoint后的交易数据：

In [15]:
# lable = pd.read_csv(LBL,engine='python')
# lable.drop(columns=['Unnamed: 0'],inplace=True)
trpt = pd.read_csv( TP, engine='python')
trsc = trsc.merge(trpt,on='store_id',suffixes=('','_trpt'))

In [16]:
trsc['pt'] = pd.to_datetime(trsc.pt, format='%Y%m%d')
trsc['pt_trpt'] = pd.to_datetime(trsc.pt_trpt, format='%Y%m%d')

In [17]:
import datetime

trsc['pt_delta'] =  trsc['pt_trpt']-trsc['pt']
trsc = trsc[(trsc[['pt_delta']]>datetime.timedelta(days=0)).all(axis=1)]
trsc.drop(columns=['pt_delta'],inplace=True)

## 特征5： 日均交易额\ 单店日均交易额在该行业占比

#operate_state 

#daily_amount

In [18]:
mch = pd.read_csv(M,engine='python',sep=' ')
# mch

In [19]:
trsc = trsc.merge(mch,on='merchant_id')

In [20]:
store_amount_sum = trsc.groupby(['store_id']).sum().reset_index()
mch_amount_sum = trsc.groupby(['merchant_id']).sum().reset_index()

In [21]:
trsc = trsc.merge(store_amount_sum[['store_id','original_amount']],on='store_id',suffixes=('','_store_sum'))
trsc = trsc.merge(mch_amount_sum[['merchant_id','original_amount']],on='merchant_id',suffixes=('','_mch_sum'))

In [22]:
trsc['days'] = trsc['pt_trpt']-datetime.datetime.strptime('2018-06-01', '%Y-%m-%d')
trsc['days'] = trsc['days'].dt.days
type(trsc.days[1])

numpy.int64

In [23]:
trsc['operate_state'] = trsc['original_amount_store_sum']/trsc['original_amount_mch_sum']
trsc['daily_amount'] = trsc['original_amount_store_sum']/trsc['days']

In [24]:
eigen = trsc.drop(columns=['merchant_id','pt','pt_trpt','days','original_amount_store_sum','original_amount_mch_sum'])

In [25]:
eigen.to_csv(EIGEN)

In [26]:
# trpt.sort_values('pt')